In [22]:
import pandas as pd
import json
import glob
import os

In [2]:
import matplotlib.pyplot as plt

In [64]:
!ls ../sampling_experiment/results_sampling/

dsmpling-run-1.txt dsmpling-run-4.txt sampling-run-2.txt sampling-run-5.txt
dsmpling-run-2.txt dsmpling-run-5.txt sampling-run-3.txt
dsmpling-run-3.txt sampling-run-1.txt sampling-run-4.txt


In [65]:
!open ../sampling_experiment/results_sampling

In [76]:
import re
modes ='[<SamplingMode.FIRST_FILE|FIRST_ROWS: 9>, <SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS: 11>, <SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS: 25>, <SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FIRST_ROWS: 27>, <SamplingMode.ALL_FILES|FIRST_ROWS: 65>, <SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS: 67>]'
sampling_modes = list(map(lambda line: re.sub('[\[\]:0-9 <>]', '', line), modes.split(',')))


In [75]:
schemas = '''(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)
(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])
(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)'''.split('\n')

schemas

['(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)',
 '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)',
 '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])',
 '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])',
 '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)',
 '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)']

In [79]:
schema_dict = dict(zip(sampling_modes, schemas))

In [86]:
schema_dict

{'SamplingMode.FIRST_FILE|FIRST_ROWS': '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)',
 'SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS': '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)',
 'SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS': '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])',
 'SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FIRST_ROWS': '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])',
 'SamplingMode.ALL_FILES|FIRST_ROWS': '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)',
 'SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS': '(Option[i64],

In [84]:
len(schemas)

6

In [85]:
len(set(schema_dict.values()))

4

In [89]:
path = '../sampling_experiment/results_sampling/dsmpling-run-1.txt'

all_dfs = pd.DataFrame()
paths = glob.glob('../sampling_experiment/results_sampling/dsmpling-run-*.txt')
for path in paths:
    with open(path) as fp:
        lines = fp.readlines()
        run = int(os.path.basename(path).split('-')[-1].replace('.txt', ''))
        data = list(filter(lambda line: line.startswith("[{'sampling_mode':"), lines))[0]
        data = eval(data)
        df = pd.DataFrame(data)
        df['sampling_time'] = df['metrics'].apply(lambda x: x['sampling_time_s'])
        df['run'] = run
        all_dfs = pd.concat((all_dfs, df))
        
        # find the Lambda Stage result, should be always the same...
        needle = 'LAMBDA Stage:'
        f_lines = list(filter(lambda line: needle in line, lines))
        for L in f_lines:
            L = L[L.find('LAMBDA'):].strip()
            print(L)
            
        # find the Lambda Stage result, should be always the same...
        needle = 'Majority detected row type is: '
        f_lines = list(filter(lambda line: needle in line, lines))
        for L in f_lines:
            L = L[L.find(needle) + len(needle):].strip()
            print(L.strip())
            
        
        needle = 'LAMBDA paths rows took:'
        f_lines = list(filter(lambda line: needle in line, lines))
        for L in f_lines:
            L = L[L.find(needle) + len(needle):].strip()
            print(L.strip())
        
        
df = all_dfs

LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
LAMBDA Stage: 18889169 rows, 0 exceptions
(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)
(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64],Option[f64])
(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)
(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Op

In [61]:
df

,sampling_mode,job_time,metrics,sampling_time,run
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.474317,"{'logical_optimization_time_s': 0.0137463, 'll...",0.775935,4
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,29.878893,"{'logical_optimization_time_s': 0.0224654, 'll...",1.150490,4
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,8.316061,"{'logical_optimization_time_s': 0.0185247, 'll...",0.964578,4
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.159707,"{'logical_optimization_time_s': 0.0394885, 'll...",1.418880,4
4,SamplingMode.ALL_FILES|FIRST_ROWS,20.294357,"{'logical_optimization_time_s': 0.340269, 'llv...",10.620800,4
5,SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS,52.301021,"{'logical_optimization_time_s': 0.60232, 'llvm...",17.458100,4
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.456474,"{'logical_optimization_time_s': 0.0137409, 'll...",0.882812,5
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,30.659847,"{'logical_optimization_time_s': 0.0223129, 'll...",1.137620,5
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,16.809237,"{'logical_optimization_time_s': 0.0201565, 'll...",0.932661,5
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.587621,"{'logical_optimization_time_s': 0.0394833, 'll...",1.456500,5


In [62]:
df['job_time_excl_sampling'] = df['job_time'] - df['sampling_time']
gdf = df.groupby('sampling_mode').agg(['mean', 'std']).reset_index().sort_values(by=('job_time', 'mean'))

gdf['sampling_mode'] = gdf['sampling_mode'].apply(lambda x: x.replace('SamplingMode.', '').replace('|', ', '))
gdf

sampling_mode   job_time             \
                                                      mean        std   
5  LAST_FILE, FIRST_FILE, LAST_ROWS, FIRST_ROWS   9.793195   0.592746   
4             LAST_FILE, FIRST_FILE, FIRST_ROWS  10.947242   3.580902   
0                         ALL_FILES, FIRST_ROWS  26.457449  12.499413   
3             FIRST_FILE, LAST_ROWS, FIRST_ROWS  31.399227   1.143487   
2                        FIRST_FILE, FIRST_ROWS  31.973531   2.130112   
1              ALL_FILES, LAST_ROWS, FIRST_ROWS  52.304517   0.409076   

  sampling_time            run           job_time_excl_sampling             
           mean       std mean       std                   mean        std  
5      1.457864  0.027975  3.0  1.581139               8.335331   0.605061  
4      0.966082  0.052034  3.0  1.581139               9.981160   3.612231  
0     10.850880  0.209241  3.0  1.581139              15.606569  12.521241  
3      1.152588  0.021711  3.0  1.581139              30.246639   1.130792  
2      0.799810  0.063635  3.0  1.581139              31.173721   2.104206  
1     17.802400  0.607483  3.0  1.581139              34.502117   0.501247

What's the difference between the two ALL files?

 'SamplingMode.ALL_FILES|FIRST_ROWS': '(i64,i64,i64,str,i64,i64,i64,i64,Option[f64],i64,Option[f64],f64,f64,null,null,null,null,null)'
 
 'SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS': '(Option[i64],Option[i64],i64,str,Option[i64],Option[i64],Option[i64],Option[i64],Option[f64],Option[i64],Option[f64],Option[f64],Option[f64],null,null,null,null,null)'

In [63]:
df

,sampling_mode,job_time,metrics,sampling_time,run,job_time_excl_sampling
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.474317,"{'logical_optimization_time_s': 0.0137463, 'll...",0.775935,4,30.698382
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,29.878893,"{'logical_optimization_time_s': 0.0224654, 'll...",1.150490,4,28.728403
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,8.316061,"{'logical_optimization_time_s': 0.0185247, 'll...",0.964578,4,7.351483
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.159707,"{'logical_optimization_time_s': 0.0394885, 'll...",1.418880,4,8.740827
4,SamplingMode.ALL_FILES|FIRST_ROWS,20.294357,"{'logical_optimization_time_s': 0.340269, 'llv...",10.620800,4,9.673557
5,SamplingMode.ALL_FILES|LAST_ROWS|FIRST_ROWS,52.301021,"{'logical_optimization_time_s': 0.60232, 'llvm...",17.458100,4,34.842921
0,SamplingMode.FIRST_FILE|FIRST_ROWS,31.456474,"{'logical_optimization_time_s': 0.0137409, 'll...",0.882812,5,30.573662
1,SamplingMode.FIRST_FILE|LAST_ROWS|FIRST_ROWS,30.659847,"{'logical_optimization_time_s': 0.0223129, 'll...",1.137620,5,29.522227
2,SamplingMode.LAST_FILE|FIRST_FILE|FIRST_ROWS,16.809237,"{'logical_optimization_time_s': 0.0201565, 'll...",0.932661,5,15.876576
3,SamplingMode.LAST_FILE|FIRST_FILE|LAST_ROWS|FI...,10.587621,"{'logical_optimization_time_s': 0.0394833, 'll...",1.456500,5,9.131121
